In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="aLhSohlVxTqiVkL9ncKr")
project = rf.workspace("tapisangdataset-yyze5").project("dataset-pisang-13sgh")
version = project.version(7)
dataset = version.download("yolov8-obb")


loading Roboflow workspace...
loading Roboflow project...


In [3]:
import os

print("Dataset location:", dataset.location)
print("Folders inside:")
print(os.listdir(dataset.location))

Dataset location: C:\Users\samue\Dataset-Pisang-7
Folders inside:
['data.yaml', 'README.dataset.txt', 'README.roboflow.txt', 'test', 'train', 'valid']


In [4]:
import os
import yaml

# Path ke file data.yaml yang diunduh
data_yaml_path = os.path.join(dataset.location, "data.yaml")

# Baca isinya
with open(data_yaml_path, "r") as f:
    data = yaml.safe_load(f)

print(data)


{'train': 'train/images', 'val': 'valid/images', 'test': 'test/images', 'names': {0: 'cordana', 1: 'fusarium', 2: 'healthy', 3: 'sigatoka'}}


In [5]:
!pip install ultralytics roboflow opencv-python


In [6]:
import os
import yaml

# Get class names from data.yaml
data_yaml_path = os.path.join(dataset.location, "data.yaml")
with open(data_yaml_path) as f:
    data = yaml.safe_load(f)
    class_names = {i: name for i, name in enumerate(data["names"])}
    print("All Class Names:", class_names)

# Inisialisasi counter untuk semua kelas
class_counts = {name: 0 for name in class_names.values()}
annotations_dir = os.path.join(dataset.location, "train")

# Hitung jumlah tiap kelas dari file label
labels_dir = os.path.join(annotations_dir, "labels")
for txt_file in os.listdir(labels_dir):
    txt_path = os.path.join(labels_dir, txt_file)
    try:
        with open(txt_path) as f:
            for line in f:
                if line.strip():
                    class_id = int(line.split()[0])
                    class_name = class_names.get(class_id, '')
                    if class_name in class_counts:
                        class_counts[class_name] += 1
    except FileNotFoundError:
        continue

print("\nClass Counts (All Classes):")
for name, count in sorted(class_counts.items()):
    print(f"{name}: {count}")


All Class Names: {0: 0, 1: 1, 2: 2, 3: 3}

Class Counts (All Classes):
0: 1223
1: 806
2: 1094
3: 1114


In [3]:
from ultralytics import YOLO

print("\n--- Starting YOLO Training with Early Stopping ---")
model = YOLO('yolov8n.pt')

model.train(
    data="C:/Users/samue/Dataset-Pisang-7/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name='motorcycle-yolo',
    patience=3  # <-- stop training jika val loss tidak membaik selama 10 epoch
)



--- Starting YOLO Training with Early Stopping ---
Ultralytics 8.3.158  Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/samue/Dataset-Pisang-7/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=motorcycle-yolo18, nbs=64, nms=False, opset=None, optimize=False

train: Scanning C:\Users\samue\Dataset-Pisang-7\train\labels.cache... 3752 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3752/3752 [00:00<?, ?it/s]

train: C:\Users\samue\Dataset-Pisang-7\train\images\387_jpg.rf.5568c7cedf232729e91ad5b2a4466588.jpg: 1 duplicate labels removed
train: C:\Users\samue\Dataset-Pisang-7\train\images\387_jpg.rf.b7bed0a0748c55d1070b7fcb203ec631.jpg: 1 duplicate labels removed
train: C:\Users\samue\Dataset-Pisang-7\train\images\cordana112_jpeg.rf.17bd3f91b6a9cf504e3b00a8a0b2a956.jpg: 1 duplicate labels removed


val: Fast image access  (ping: 0.20.1 ms, read: 26.226.4 MB/s, size: 51.5 KB)


val: Scanning C:\Users\samue\Dataset-Pisang-7\valid\labels.cache... 1065 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1065/1065 [00:00<?, ?it/s]

...

                   all       1065       1192      0.957      0.913       0.96      0.907


EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 29, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

32 epochs completed in 0.697 hours.
Optimizer stripped from runs\detect\motorcycle-yolo18\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\motorcycle-yolo18\weights\best.pt, 6.2MB

Validating runs\detect\motorcycle-yolo18\weights\best.pt...
Ultralytics 8.3.158  Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:12<00:00,  2.79it/s]


                   all       1065       1192      0.947      0.921      0.959       0.91
               cordana        189        310      0.847      0.716      0.868       0.78
              fusarium        261        261      0.965      0.989      0.987      0.907
               healthy        291        294      0.981       0.99      0.994       0.97
              sigatoka        323        327      0.995      0.988      0.987      0.984
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs\detect\motorcycle-yolo18


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000029C73F31390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

In [5]:
import cv2
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load model hasil training
model = YOLO("runs/detect/motorcycle-yolo18/weights/best.pt")

# Path ke folder gambar yang ingin diprediksi
image_folder = "Dataset-Pisang-7/test/images"  # Ganti dengan folder lain jika perlu

# Ambil semua file gambar
# Mengganti 'test_image_folder' dengan 'image_folder'
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Loop untuk prediksi semua gambar dan tampilkan hasilnya
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file) # Mengganti 'test_image_folder' dengan 'image_folder'

    # Jalankan prediksi
    results = model(image_path)

    # Tampilkan hasil deteksi menggunakan OpenCV/Matplotlib
    result_image = results[0].plot()  # Hasil prediksi dengan bounding box (numpy array BGR)
    result_image_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 8))
    plt.imshow(result_image_rgb)
    plt.title(f"Deteksi: {image_file}")
    plt.axis('off')
    plt.show()


image 1/1 C:\Users\samue\Dataset-Pisang-7\test\images\0_jpg.rf.0e188e31c6b2a793b8e4bccd309c847f.jpg: 640x640 1 cordana, 58.0ms
Speed: 11.6ms preprocess, 58.0ms inference, 7.3ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x800 with 1 Axes>


image 1/1 C:\Users\samue\Dataset-Pisang-7\test\images\102_jpg.rf.0f12e4f48c9fdb096ef01e85a27f89a2.jpg: 640x640 1 cordana, 57.2ms
Speed: 7.9ms preprocess, 57.2ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x800 with 1 Axes>


image 1/1 C:\Users\samue\Dataset-Pisang-7\test\images\102_jpg.rf.50c220f49a87d3fb7672b430d70e903a.jpg: 640x640 1 cordana, 57.4ms
Speed: 7.7ms preprocess, 57.4ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)


...

<Figure size 800x800 with 1 Axes>


image 1/1 C:\Users\samue\Dataset-Pisang-7\test\images\Image_99_jpg.rf.d36d0020fd243c81049b55acecac3b66.jpg: 640x640 1 fusarium, 23.6ms
Speed: 5.4ms preprocess, 23.6ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x800 with 1 Axes>


image 1/1 C:\Users\samue\Dataset-Pisang-7\test\images\Image_99_jpg.rf.e83a301a12ef802949de2c97d7636ce7.jpg: 640x640 1 healthy, 23.5ms
Speed: 5.3ms preprocess, 23.5ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x800 with 1 Axes>

In [ ]:
output_folder = "hasil_deteksi"
os.makedirs(output_folder, exist_ok=True)

for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    results = model(image_path)

    # Ambil hasil dan simpan sebagai file
    result_image = results[0].plot()
    save_path = os.path.join(output_folder, image_file)
    cv2.imwrite(save_path, result_image)

print(f"Hasil deteksi disimpan di folder: {output_folder}")
